# first attempt - simple reverse prediction

In [15]:
import numpy as np
import pandas as pd
from functions import *
from read_from_file_df import *
import pickle
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [16]:
gen = 2
name_df = f'{PATH_DF}\\{SIZE}size_{AMOUNT_BOARDS}boards_{gen}gen_reverse'
reverse_df = pd.read_pickle(f'{name_df}.pkl')

In [17]:
new_columns = [f'Col_{i}' for i in range(1, gen*SIZE*SIZE + 1)]
reverse_df_sort = reverse_df.sort_values(by = new_columns).reset_index(drop=True)
for i in reverse_df_sort.columns:
    reverse_df_sort[i] = reverse_df_sort[i].astype(int)

In [18]:
print("reverse df:", len(reverse_df))
print("reverse df sort:",len(reverse_df_sort))

reverse df: 29760
reverse df sort: 29760


In [19]:
# Step 1: Prepare Data
amount_features = len(reverse_df_sort.columns) - SIZE*SIZE #the previous boards
features = reverse_df_sort.iloc[:, :amount_features]
name_col = 'Col_' + str(amount_features + 1)  # Target: the first pixel in the board
target = reverse_df_sort[name_col]

# Step 2: Split Data
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.1, random_state=365)

print("len x train: ", len(X_train))
print("len x test: ",len(X_test))
print("len y train: ",len(y_train))
print("len y test: ",len(y_test))

dt = tree.DecisionTreeClassifier(random_state=42)
dt = dt.fit(X_train, y_train)

y_train_pred = dt.predict(X_train)
y_test_pred = dt.predict(X_test)

train_test_full_error = pd.concat([measure_error(y_train, y_train_pred, 'train'),
                                measure_error(y_test, y_test_pred, 'test')],
                                axis=1)

print(dt.tree_.node_count, dt.tree_.max_depth)
train_test_full_error

len x train:  26784
len x test:  2976
len y train:  26784
len y test:  2976
639 12


,train,test
accuracy,1.0,1.0
precision,1.0,1.0
recall,1.0,1.0
f1,1.0,1.0


In [20]:
reverse_df_sort.iloc[21540].values

array([1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 0, 0, 0, 1])

In [21]:
features.iloc[21540].values

array([1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1])

In [22]:
target.iloc[21540]

0

In [23]:
depth_to_check = [4,5,6,7,8,9,10,15,20,30,50,100]

for i in depth_to_check:
    dt, node_count, max_depth, train_test_full_error  = dec_tree_df(X_train,y_train, X_test, y_test, i, 42)
    
    name_file = f"{PATH_MODELS}\\reverse_model1\\decision_tree_reverse_df_model_{str(i)}.pkl"
    with open(name_file, 'wb') as file:
        pickle.dump(dt, file)
    
    print(node_count, max_depth)
    print(train_test_full_error)

31 4
              train      test
accuracy   0.725956  0.737567
precision  0.601863  0.612551
recall     0.447989  0.474630
f1         0.513650  0.534842
63 5
              train      test
accuracy   0.734058  0.746976
precision  0.670305  0.694165
recall     0.347781  0.364693
f1         0.457956  0.478170
123 6
              train      test
accuracy   0.763217  0.775874
precision  0.679851  0.690834
recall     0.504623  0.533827
f1         0.579276  0.602266
227 7
              train      test
accuracy   0.794093  0.805780
precision  0.692430  0.699134
recall     0.652335  0.682875
f1         0.671785  0.690909
393 8
              train      test
accuracy   0.872760  0.861223
precision  0.803753  0.777894
recall     0.801896  0.788584
f1         0.802823  0.783202
597 9
              train      test
accuracy   0.992981  0.993280
precision  0.983215  0.987368
recall     0.995261  0.991543
f1         0.989202  0.989451
621 10
              train      test
accuracy   0.996154  0.997312

In [24]:
amount_features = len(reverse_df_sort.columns) - SIZE*SIZE #the previous boards
features = reverse_df_sort.iloc[:, :amount_features]
for i in range(SIZE*SIZE): # to any pixel in the expected board
    name_col = 'Col_' + str(i+amount_features + 1)
    target = reverse_df_sort[name_col]
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.1, random_state=613)
    dt, node_count, max_depth, train_test_full_error  = dec_tree_df(X_train,y_train, X_test, y_test, 10, 42)
    name_file = f"{PATH_MODELS}\\reverse1\\dt_r1_pixel_{str(i+1)}.pkl"
    with open(name_file, 'wb') as file:
        pickle.dump(dt, file)
    print(i)
    print(node_count, max_depth)
    print(train_test_full_error)

0
631 10
              train      test
accuracy   0.986410  0.983535
precision  0.984105  0.973712
recall     0.973636  0.974737
f1         0.978842  0.974224
1
701 10
              train      test
accuracy   0.976366  0.969758
precision  0.951059  0.945361
recall     0.975089  0.961216
f1         0.962924  0.953222
2
647 10
              train      test
accuracy   0.986634  0.982863
precision  0.980711  0.974542
recall     0.977511  0.973550
f1         0.979108  0.974046
3
637 10
              train      test
accuracy   0.992047  0.988575
precision  0.982903  0.979058
recall     0.991909  0.985248
f1         0.987385  0.982143
4
625 10
              train      test
accuracy   0.996789  0.995632
precision  0.995615  0.993651
recall     0.994200  0.992600
f1         0.994907  0.993125
5
605 10
              train      test
accuracy   0.998880  0.998656
precision  0.998023  0.996875
recall     0.998487  0.998956
f1         0.998255  0.997914
6
615 10
              train      test
accurac